In [64]:
from pyramid.arima import auto_arima
import pandas as pd
import numpy as np

In [65]:
data = pd.read_csv('/home/affine/Downloads/Finance_Final_data.csv')

In [66]:
data.drop(data.tail(2).index, inplace=True)

In [67]:
data.shape

(132, 4298)

In [68]:
data.columns

Index(['year', 'month', 'High000075.KS', 'Low000075.KS', 'Open000075.KS',
       'Close000075.KS', 'High000157.KS', 'Low000157.KS', 'Open000157.KS',
       'Close000157.KS',
       ...
       'OpenWHE.BE', 'CloseWHE.BE', 'HighWHRL3.SA', 'LowWHRL3.SA',
       'OpenWHRL3.SA', 'CloseWHRL3.SA', 'High^IXIC', 'Low^IXIC', 'Open^IXIC',
       'Close^IXIC'],
      dtype='object', length=4298)

In [69]:
DV = 'High^IXIC'

In [70]:
data['High^IXIC'].isna().sum()

0

In [71]:
data.shape

(132, 4298)

In [72]:
train = data[DV][0:100]
test = data[DV][100:]

In [73]:
print(train.isna().sum())
print(test.isna().sum())

0
0


In [74]:
stepwise_model = auto_arima(train, start_p=1, start_q=1,
                           max_p=3, max_q=3, m=12,
                           start_P=0, seasonal=True,
                           d=1, D=1, trace=True,
                           error_action='ignore',  
                           suppress_warnings=True, 
                           stepwise=True)

Fit ARIMA: order=(1, 1, 1) seasonal_order=(0, 1, 1, 12); AIC=1101.647, BIC=1113.977, Fit time=1.040 seconds
Fit ARIMA: order=(0, 1, 0) seasonal_order=(0, 1, 0, 12); AIC=1135.094, BIC=1140.026, Fit time=0.022 seconds
Fit ARIMA: order=(1, 1, 0) seasonal_order=(1, 1, 0, 12); AIC=1112.940, BIC=1122.804, Fit time=0.823 seconds
Fit ARIMA: order=(0, 1, 1) seasonal_order=(0, 1, 1, 12); AIC=1100.371, BIC=1110.235, Fit time=0.721 seconds
Fit ARIMA: order=(0, 1, 1) seasonal_order=(1, 1, 1, 12); AIC=1102.355, BIC=1114.685, Fit time=0.967 seconds
Fit ARIMA: order=(0, 1, 1) seasonal_order=(0, 1, 0, 12); AIC=1132.449, BIC=1139.847, Fit time=0.134 seconds
Fit ARIMA: order=(0, 1, 1) seasonal_order=(0, 1, 2, 12); AIC=nan, BIC=nan, Fit time=nan seconds
Fit ARIMA: order=(0, 1, 1) seasonal_order=(1, 1, 2, 12); AIC=nan, BIC=nan, Fit time=nan seconds
Fit ARIMA: order=(0, 1, 0) seasonal_order=(0, 1, 1, 12); AIC=1101.657, BIC=1109.055, Fit time=0.368 seconds
Fit ARIMA: order=(0, 1, 2) seasonal_order=(0, 1, 1, 

In [75]:
print(stepwise_model.aic())

1100.3710263101848


In [76]:
stepwise_model.fit(train)

ARIMA(callback=None, disp=0, maxiter=50, method=None, order=(0, 1, 1),
   out_of_sample_size=0, scoring='mse', scoring_args={},
   seasonal_order=(0, 1, 1, 12), solver='lbfgs', start_params=None,
   suppress_warnings=True, transparams=True, trend='c')

In [77]:
test.shape

(32,)

In [78]:
future_forecast = stepwise_model.predict(n_periods=32)

In [79]:
type(test)

pandas.core.series.Series

In [80]:
test_data = pd.DataFrame({'actual': test, 'predicted': future_forecast})

In [81]:
test_data

actual    predicted
100  5050.212524  5072.684807
101  5096.210427  5127.653725
102  5105.622736  5229.835584
103  4984.817150  5287.728566
104  4790.483817  5393.179997
105  4895.230935  5423.575172
106  5103.614990  5497.554720
107  5072.878662  5598.056199
108  4662.806821  5680.783504
109  4499.422510  5779.266884
110  4770.360485  5872.818414
111  4916.229074  5968.033374
112  4808.790016  6053.422820
113  4878.533181  6120.281694
114  5035.984985  6234.353509
115  5231.513481  6304.136448
116  5273.439965  6421.477836
117  5279.340960  6463.762968
118  5285.543318  6549.632472
119  5437.372861  6662.023907
120  5572.404956  6756.641169
121  5774.061575  6867.014505
122  5883.044391  6972.455991
123  5932.016884  7079.560908
124  6137.398193  7176.840310
125  6251.044545  7255.589141
126  6312.877490  7381.550913
127  6337.219578  7463.223808
128  6444.590918  7592.455153
129  6617.148171  7646.630241
130  6810.008580  7744.389702
131  6917.870947  7868.671094

In [82]:
test_data['APE'] = abs(test_data['actual'] - test_data['predicted'])/test_data['actual']

In [83]:
print('Mape is :', np.mean(test_data['APE']))

Mape is : 0.16833052854858988
